In [5]:
from tqdm import tqdm
from importlib import reload  # Python 3.4+

import pandas as pd
import numpy as np
import time

import lib.config as cfg
import lib.preproc as preproc
import lib.database as db
import lib.ts_util as ts_util

In [3]:
con = db.connect_psql()

Database: 
Username: postgres


Password:  ········


## Requires: import data to postgres
- create tables/views: 
    - view_lab_creatinine_summary
    - table_ts_lab
    - table_lab_stats

## Step 0: Normalization
- Calcuate normalization

In [ ]:
df_selected_lab = pd.read_sql_query(""" SELECT * FROM table_lab_stats""", con = con)
df_selected_lab['mean'] = (df_selected_lab['val_max_mean'] + df_selected_lab['val_min_mean'])/2
df_selected_lab['std'] = (df_selected_lab['val_max_std'] + df_selected_lab['val_min_std'])/2

In [ ]:
# Load all pid with 20+ creatinine measurements
query = """
SELECT DISTINCT
    patientunitstayid
FROM table_ts_lab
"""

df_pid = pd.read_sql_query(query, con)
print('sample size: ', df_pid.shape)

## Step 1: Calculate AKI label
- run time: ~20 min
- Calculate AKI label using sparse time series
- Rule
    - Rule 1: 1.5x baseline creatinine in 7 days
    - Rule 2: 0.3mg/dL creatinine increase in 48h

In [ ]:
query = """
SELECT
	*
FROM table_ts_lab
WHERE labname = 'creatinine'
ORDER BY patientunitstayid, tsid
"""
df_ts_label = pd.read_sql_query(query, con)
df_ts_label['aki'] = 0
pid_list = list(np.unique(df_ts_label['patientunitstayid']))

In [ ]:
# this takes ~ 20 minutes to run
for pid in tqdm(pid_list, total=len(pid_list)):
    df_ts_patient = df_ts_label[df_ts_label['patientunitstayid'] == pid]
    
    for i in range(1, len(df_ts_patient)):
        dt_current = df_ts_patient.iloc[i]
        ts_current = dt_current['tsid']
        df_ts_patient_past = df_ts_patient.iloc[0:i]

        # current row index in master label table
        idx = dt_current.name

        # Rule #1: 1.5x baseline creatinine in 7 days
        df_temp = df_ts_patient_past[df_ts_patient_past['tsid'] > ts_current - 7*4]
        if len(df_temp):
            # creat_baseline = min(df_temp['value_min']) # baseline = 7-day min
            # creat_baseline = np.median(df_temp['value_min']) # baseline = 7-day median
            creat_baseline = np.percentile(df_temp['value_min'], 25) # baseline = 7-day 25 percentile
            # creat_baseline = df_temp['value_min'].iloc[0] # baseline = 7-day start
            if dt_current['value_max'] >= creat_baseline*1.5:
                df_ts_label.loc[idx, 'aki'] = 1

        # Rule #2: 0.3mg/dL creatinine increase in 48h
        df_temp = df_ts_patient_past[df_ts_patient_past['tsid'] > ts_current - 2*4]
        if len(df_temp):
            creat_baseline = np.median(df_temp['value_min'])
            if dt_current['value_max'] >= creat_baseline + 0.3:
                df_ts_label.loc[idx, 'aki'] = 1

df_ts_label.to_csv('./df_ts_label.csv', index=False)

In [ ]:
# Under baseline = 7d median def: ~ 20% of sparse ts are labeled as 'aki positive'
sum(df_ts_label['aki'])/len(df_ts_label)

## Step 2: Prepare trainig dataset
- run time: CPU heavy, ~2.5h
- Use label: baseline creatinine use median
- Function
    - calculate categorical variables
    - calculate lab variables
    - fill sparse time series to dense time series
    - calculate multi-head gt_label
    - calculate multi-head aux tasks

In [ ]:
df_ts_label_all = pd.read_csv('./df_ts_label_median.csv')
df_ts_label_all

In [ ]:
# patient demographic
query = """
SELECT
    patient.patientunitstayid,
    patient.gender,
    patient.age,
    patient.ethnicity, -- Caucasian, African American
    patient.hospitaladmitsource -- Other Hospital, Emergency Department, Operating Room, Floor
FROM patient patient
"""
df_patient = pd.read_sql_query(query, con)
df_patient = df_pid.merge(df_patient, how='inner')

# Drop rows with invalid gender
df_patient = df_patient[df_patient['gender'] != '']
df_patient = df_patient[df_patient['gender'] != 'Unknown']

# Convert str age -> numerical age
# patients age greater than 89 are expressed as '>89'
df_patient['age'] = list(map(lambda x: int(x.replace('> ', '')), df_patient['age']))

# map catigorical to numerical: gender
df_patient['gender_female'] = list(map(lambda x: 0 if x == 'Male' else 1, df_patient['gender']))

# Drop patient age <= 18
df_patient = df_patient[df_patient['age'] > 18]

df_patient['ethnicity_Caucasian'] = (df_patient['ethnicity'] == 'Caucasian').astype(int)
df_patient['ethnicity_Black'] = (df_patient['ethnicity'] == 'African American').astype(int)
df_patient['admission_Emergency'] = (df_patient['hospitaladmitsource'] == 'Emergency Department').astype(int)
df_patient['admission_Floor'] = (df_patient['hospitaladmitsource'] == 'Floor').astype(int)
df_patient['admission_Surgical'] = (df_patient['hospitaladmitsource'] == 'Operating Room').astype(int)
df_patient['admission_Transfer'] = (df_patient['hospitaladmitsource'] == 'Other Hospital').astype(int)

df_patient = df_patient.drop(['ethnicity', 'hospitaladmitsource', 'gender'], axis=1)

In [ ]:
# reload preproc library
preproc = reload(preproc)

### TBD
- error pid=449988 / current # ts = 31
- error: pid=2385766 / loop don't terminate


In [ ]:
list_pid = df_pid['patientunitstayid']
lab_columns = list(df_selected_lab['labname'])

df_lab_agg = pd.DataFrame()

for pid in tqdm(list_pid, total=len(list_pid)):
    try:
        df_timestamp_lab = preproc.get_df_timestamp_by_patient(pid, con)
        df_timestamp_lab = df_timestamp_lab.rename(columns={'labname': 'variable'})

        df_lab = preproc.calc_df_ts_from_timestamp(df_timestamp_lab, lab_columns)

        # merge label into df_predictor
        df_ts_label = df_ts_label_all[df_ts_label_all['patientunitstayid'] == pid][['patientunitstayid', 'tsid', 'aki']]
        df_ts_label = df_lab[['patientunitstayid', 'tsid']].merge(df_ts_label, how='left')

        list_label = df_ts_label['aki'].copy()
        list_label = preproc.calc_list_label(list_label)

        # label
        df_lab['label_aki_gt'] = list_label
        df_lab['label_aki_36h'] = preproc.calc_label_shift(list_label, 6)
        df_lab['label_aki_24h'] = preproc.calc_label_shift(list_label, 4)
        df_lab['label_aki_12h'] = preproc.calc_label_shift(list_label, 2)
        df_lab['label_aki_6h'] = preproc.calc_label_shift(list_label, 1)

        # aux task
        df_aux_task = preproc.get_df_aux_task(df_lab)
        df_lab = df_lab.join(df_aux_task)

        df_lab_agg = df_lab_agg.append(df_lab.copy(), ignore_index=True)
    except:
        print('error: ', pid)

In [ ]:
df_train = df_patient.merge(df_lab_agg)
df_train.to_csv('./df_train.csv', index=False) # 700M
df_train.to_pickle('df_train.pkl') # 3.7G

In [ ]:
df_train.shape

## Step 3: Build trainig dataset
- train/test split
- define label/data columns
- Missing value impute with -1
- Normalization


## Step 3.1: train size = 8500
- memory constraint

## Step 3.2: process training set

In [ ]:
df_train = pd.read_pickle('df_train_median.pkl') # this is much faster than read from csv
pid_list = np.unique(df_train['patientunitstayid'])
len(df_train)

In [ ]:
# def columns
col_all = df_train.columns
col_meta = ['patientunitstayid', 'tsid']
col_label = list(col_all[list(map(lambda x: 'label_' in x, col_all))])
col_label_used = col_label[1:] # remove gt_column
col_regres = list(col_all[list(map(lambda x: 'regres_' in x, col_all))])
col_to_drop = col_meta + col_label + col_regres

# def max time series length
max_ts_length = 128

# build model cfg
model_config = {
    'col_to_drop': col_to_drop,
    'col_label': col_label,
    'col_regres': col_regres,
    'max_ts_length': max_ts_length,
}

# Missing value impute with -1
df_train = df_train.fillna(-1)

# Normalization
col_lab_min = list(map(lambda x: x+'_min', cfg.selected_lab))
col_lab_max = list(map(lambda x: x+'_max', cfg.selected_lab))
col_require_norm = col_lab_min + col_lab_max + col_regres

for col in col_require_norm:
    v_mean = np.mean(df_train[col])
    v_std = np.std(df_train[col])

    df_train[col] = (df_train[col] - v_mean)/v_std

## [TBD] Step 3.3: process testing set

## Step 4: Build Fixed Size Tensor
- Convert 2D flattened ts table: [pid, tsid], [feature]
- To 3D tensor: [pid], [tsid], [feature]

In [ ]:
list_tensor_data = []
list_tensor_label = []
list_tensor_regres = []

for pid in pid_list:
    tensor_data, tensor_label, tensor_regres = ts_util.get_ts_by_pid(df_train, pid, model_config)

    tensor_data = ts_util.build_fixed_length_ts(tensor_data, max_ts_length)
    tensor_label = ts_util.build_fixed_length_ts(tensor_label, max_ts_length)
    tensor_regres = ts_util.build_fixed_length_ts(tensor_regres, max_ts_length)

    list_tensor_data.append(tensor_data)
    list_tensor_label.append(tensor_label)
    list_tensor_regres.append(tensor_regres)

tensor_data_agg = torch.cat(list_tensor_data, dim=1)
tensor_label_agg = torch.cat(list_tensor_label, dim=1)
tensor_regres_agg = torch.cat(list_tensor_regres, dim=1)
